1. รหัส ชื่อ และหมู่ของนิสิตในกลุ่ม
6710451208 มนัสนันท์ รัตนเกียรติขจร 200
6710451402 สิรินดา ศิลปเจริญ 200

2. วัตถุประสงค์ของระบบต้นแบบ
- เพื่อพัฒนาพัฒนาระบบช่วยวิเคราะห์และคาดการณ์ความเสี่ยงในการลาออกของพนักงาน (Employee Attrition)

3. ลิงค์ไปยังข้อมูลที่จะใช้ในระบบต้นแบบ

In [1]:
import pandas as pd
df_employee = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
df_employee

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


4. ลิงค์ไปยัง Hugging Face Space ของตัวเองที่ออกแบบให้เหมาะสมกับวัตถุประสงค์
- https://huggingface.co/spaces/Bifern1403/Employee-Resign-Prediction/tree/main

In [1]:
import joblib
import pandas as pd
import gradio as gr

attrition_model = joblib.load("rf_selected_features_model.joblib")

model_features = [
    'StockOptionLevel','Age','EnvironmentSatisfaction','JobLevel',
    'JobRole_Laboratory Technician','MonthlyIncome','YearsAtCompany',
    'JobSatisfaction','YearsSinceLastPromotion','JobRole_Healthcare Representative',
    'BusinessTravel_Travel_Frequently','PercentSalaryHike','YearsInCurrentRole',
    'OverTime_Yes','DailyRate'
]

def predict_attrition(age, overtime, monthly_income, years_at_company, years_in_current_role,
                      job_satisfaction, env_satisfaction, job_level, business_travel):

    sample = pd.DataFrame(0, index=[0], columns=model_features)

    sample['Age'] = age
    sample['OverTime_Yes'] = 1 if overtime == "Yes" else 0
    sample['MonthlyIncome'] = monthly_income
    sample['YearsAtCompany'] = years_at_company
    sample['YearsInCurrentRole'] = years_in_current_role
    sample['JobSatisfaction'] = job_satisfaction
    sample['EnvironmentSatisfaction'] = env_satisfaction
    sample['JobLevel'] = job_level
    sample['BusinessTravel_Travel_Frequently'] = 1 if business_travel == "Travel_Frequently" else 0

    sample['StockOptionLevel'] = 0
    sample['YearsSinceLastPromotion'] = 0
    sample['PercentSalaryHike'] = 10
    sample['DailyRate'] = 500
    sample['JobRole_Laboratory Technician'] = 0
    sample['JobRole_Healthcare Representative'] = 0

    pred = attrition_model.predict(sample)[0]
    return "Yes (ลาออก)" if pred == 1 else "No (ไม่ลาออก)"

overtime = ['Yes', 'No']
business_travel = ['Non-Travel', 'Travel_Rarely', 'Travel_Frequently']

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Row():
        age = gr.Slider(label="Age", minimum=18, maximum=60, step=1)
        monthly_income = gr.Number(label="Monthly Income", minimum=0)

    with gr.Row():
        years_at_company = gr.Slider(label="Years At Company", minimum=0, maximum=40, step=1)
        years_in_current_role = gr.Slider(label="Years In Current Role", minimum=0, maximum=20, step=1)

    with gr.Row():
        job_satisfaction = gr.Slider(label="Job Satisfaction (1-4)", minimum=1, maximum=4, step=1)
        env_satisfaction = gr.Slider(label="Environment Satisfaction (1-4)", minimum=1, maximum=4, step=1)

    with gr.Row():
        job_level = gr.Slider(label="Job Level", minimum=1, maximum=5, step=1)
        overtime_input = gr.Dropdown(overtime, label="OverTime")
        business_travel_input = gr.Dropdown(business_travel, label="Business Travel")

    predict_btn = gr.Button("Predict Attrition", variant="primary")
    result = gr.Textbox(label="Prediction Result")

    inputs = [age, overtime_input, monthly_income, years_at_company, years_in_current_role,
              job_satisfaction, env_satisfaction, job_level, business_travel_input]

    predict_btn.click(predict_attrition, inputs=inputs, outputs=[result])

if __name__ == "__main__":
    demo.launch(share=True)


c:\Users\borfor\Documents\ML-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\borfor\Documents\ML-project\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.7.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\borfor\Documents\ML-project\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.7.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://e35bdce21d445e466c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


5. ก​าร​มีส่วน​ร่วมของสม​าชิกแต่ละ​คนในกลุ่ม (แต่ล​ะคนทำอะไรบ้าง)
6710451208 มนัสนันท์ รัตนเกียรติขจร 200
- ทำ code ส่วนของ UI ที่เอาไปใส่ใน hugging face space
- ปรับ model ส่วน feature engineering ใหม่
- ช่วยกันออกความคิดเห็นเกี่ยวกับการเลือกโมเดล
6710451402 สิรินดา ศิลปเจริญ 200
- ปรับ training data และ data engineering ใหม่
- ช่วยกันออกความคิดเห็นเกี่ยวกับการเลือกโมเดล
- ช่วยออกความคิดเห็นในส่วนของ code UI

6. กา​รเปิ​ดเผ​ยกา​รใช้เค​รื่อ​งมือปัญ​ญาป​ระดิษฐ์ (ใช้อะไร ใช้เพื่ออะไร ใช้อย่างไร, pro​mpt อย่างไร)
- ใช้ในการออกแบบ UI
- ใช้ในการทำความความเข้าใจ code และหาข้อมูลเพิ่มเติมเกี่ยวกับ code